<a href="https://colab.research.google.com/github/wulannurafifah/Content-Based-Recomendation/blob/main/content_based_filtering_Rekomendasi_Studi_Independen_dan_Magang.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Content Based Filtering : Sistem Rekomendasi Studi Independen dan Magang

In [20]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import re
import random

df = pd.read_csv("https://raw.githubusercontent.com/wulannurafifah/Content-Based-Recomendation/main/Data%20Rekomendasi%20Studi%20Independen%20dan%20Magang%20-%20Sheet1.csv")
df.head()

,Nama Perusahaan,Alamat,Modul Pembelajaran
0,Wulan Nur Afifah,"Jl. Cisadane, Kebondalem, Pemalang",Algoritma dan Pemrograman Bahasa Pemrograman S...
1,AI 4 Jobs PT Orbit Ventura Indonesia,"Veteran RI Building 15th Floor Unit Z15-002, P...",Logika dan Konsep Teknologi AI. Sasaran Pengem...
2,Akademi Mobile App Developer PT Nurul Fikri Ci...,Jl. Situ Indah No.116 RT. 006 RW. 010 Kel. Tug...,Soft Skill. Mampu menjelaskan pengertian komun...
3,Cloud Administrator PT MariBelajar Indonesia C...,"Jln Tunggul Ametung IVa no 8, Desa Ubung Kaja,...",Microsoft 365 Fundamentals. Target pembelajara...
4,Bisnis Kecerdasan Artifisial (AI-HUSTLER) PT B...,JLN. IR H JUANDA NO. 84 BANDUNG,"Jr. Project Manager, Mengenal teknik dan langk..."


## 1. Ikhtisar

In [21]:
df.describe()

,Nama Perusahaan,Alamat,Modul Pembelajaran
count,11,11,11
unique,11,11,11
top,Wulan Nur Afifah,"Jl. Cisadane, Kebondalem, Pemalang",Algoritma dan Pemrograman Bahasa Pemrograman S...
freq,1,1,1


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Nama Perusahaan     11 non-null     object
 1   Alamat              11 non-null     object
 2   Modul Pembelajaran  11 non-null     object
dtypes: object(3)
memory usage: 392.0+ bytes


## 2. Modul Pembelajaran (Sebelum Preprocessing)

In [23]:
def print_description(index):
    example = df[df.index == index][['Modul Pembelajaran', 'Nama Perusahaan','Alamat']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Nama:', example[1])
        print('Alamat:', example[2])

In [24]:
print_description(1)

Logika dan Konsep Teknologi AI. Sasaran Pengembangan Skill Coding Memiliki cara berpikir berdasarkan metode-metode Artificial Intelligence dan memiliki wawasan dan perkembangan tentang Artificial Intelligence. Pada  pembelajaran   individu, peserta akan mengikuti pembelajaran tentang perkembangan teknologi,  sejarah Artificial Intelligence (AI) dan  prinsip-prinsip dasar AI serta pengenalan metode-metode yang ada pada Domain AI. Metode Penelitian AI. Mampu menggunakan Analisis dan Statistik yang terdapat pada Domain  Artificial Intelligence untuk melakukan Pemodelan AI dan Analisis Masalah. Pada pembelajaran individu, peserta akan mengikuti pembelajaran tentang konsep-konsep analisis dan statistik dalam Domain AI, yaitu Data Science, Computer Vision dan Natural Language Proccessing. Siklus ProjekAI. Mampu merancang dan mengimplementasi-kan AI Cycle Project (metode-metode yang ada pada domain AI) dalam kehidupan sehari-hari. Pada pembelajaran individu, peserta akan mengikuti pembelajara

In [25]:
print_description(3)

Microsoft 365 Fundamentals. Target pembelajaran pada modul ini adalah  peserta mampu: Menjelaskan konsep awan, Menjelaskan layanan dan konsep inti Microsoft 365, Menjelaskan keamanan, kepatuhan, privasi, dan kepercayaan di Microsoft 365, Menjelaskan harga dan dukungan Microsoft 365. Microsoft Azure Fundamentals. Setelah mengikuti kursus ini, maka mahasiswa mampu:   Menjelaskan konsep cloud    Menjelaskan layanan inti Azure    Menjelaskan solusi inti dan alat manajemen di Azure    Menjelaskan keamanan umum dan fitur keamanan jaringan    Menjelaskan fitur identitas, tata kelola, privasi, dan kepatuhan    Menjelaskan manajemen biaya Azure dan Perjanjian Tingkat Layanan. Capstone Project. Peserta mengimplementasikan kompetensi teknis yang dipelajari sebagai solusi bisnis/permasalahan yang telah ditetapkan. Peserta akan didorong untuk bekerja secara berkelompok, sehingga selain kompetensi teknis, para peserta juga akan memperoleh keterampilan soft-skills, seperti keterampilan komunikasi, ko

In [26]:
print_description(7)

Pemahaman terhadap alur produksi 2D artist di dalam produksi game, serta implementasinya. Peserta mampu membuat asset 2d  sesuai dengan GDD, peserta mampu melakukan riset dan membuat, mampu memahami dan menguasai  alur produksi asset dalam game, mampu memahami prinsip animasi. Melakukan riset terkait trend desain/gambar, seperti style gambar bersama dengan Tim. Memahami dan mengetahui  cara melakukan riset untuk menentukan  tampilan visual yang menarik Bersama dengan tim, agar produk dapat terdeliver sesuai dengan GDD, peserta mampu membuat asset 2d sesuai dengan GDD, peserta mampu melakukan riset dan membuat, mampu memahami dan menguasai  alur produksi asset dalam game, mampu memahami prinsip animasi. Melakukan analisis dari sisi warna Game Designer. Implementasi, membuat  sketsa awal / dummy yang digunakan sebagai panduan visual asset secara keseluruhan. Melakukan analisis dari sisi style Game Designer. Implementasi, membuat  sketsa awal / dummy yang digunakan sebagai panduan visual 

## 3. Text Preprocessing

In [27]:
import nltk
nltk.download('stopwords')
clean_spcl = re.compile('[/(){}\[\]\|@,;]')
clean_symbol = re.compile('[^0-9a-z #+_]')
#stopworda = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = clean_spcl.sub(' ', text)
    text = clean_symbol.sub('', text)
    #text = ' '.join(word for word in text.split() if word not in stopworda) # hapus stopword dari kolom deskripsi
    return text
    
df['modulpembelajaran_clean'] = df['Modul Pembelajaran'].apply(clean_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [28]:
df.head()

,Nama Perusahaan,Alamat,Modul Pembelajaran,modulpembelajaran_clean
0,Wulan Nur Afifah,"Jl. Cisadane, Kebondalem, Pemalang",Algoritma dan Pemrograman Bahasa Pemrograman S...,algoritma dan pemrograman bahasa pemrograman s...
1,AI 4 Jobs PT Orbit Ventura Indonesia,"Veteran RI Building 15th Floor Unit Z15-002, P...",Logika dan Konsep Teknologi AI. Sasaran Pengem...,logika dan konsep teknologi ai sasaran pengemb...
2,Akademi Mobile App Developer PT Nurul Fikri Ci...,Jl. Situ Indah No.116 RT. 006 RW. 010 Kel. Tug...,Soft Skill. Mampu menjelaskan pengertian komun...,soft skill mampu menjelaskan pengertian komuni...
3,Cloud Administrator PT MariBelajar Indonesia C...,"Jln Tunggul Ametung IVa no 8, Desa Ubung Kaja,...",Microsoft 365 Fundamentals. Target pembelajara...,microsoft 365 fundamentals target pembelajaran...
4,Bisnis Kecerdasan Artifisial (AI-HUSTLER) PT B...,JLN. IR H JUANDA NO. 84 BANDUNG,"Jr. Project Manager, Mengenal teknik dan langk...",jr project manager mengenal teknik dan langka...


## 4. Modul Pembelajaran (Setelah Preprocessing)

In [29]:
# Deskripsi kedua (Setelah preprocessing)
def print_description_clean(index):
    example = df[df.index == index][['modulpembelajaran_clean', 'Nama Perusahaan','Alamat']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Nama:', example[1])
        print('Alamat:', example[2])

In [30]:
print_description_clean(1)

logika dan konsep teknologi ai sasaran pengembangan skill coding memiliki cara berpikir berdasarkan metodemetode artificial intelligence dan memiliki wawasan dan perkembangan tentang artificial intelligence pada  pembelajaran   individu  peserta akan mengikuti pembelajaran tentang perkembangan teknologi   sejarah artificial intelligence  ai  dan  prinsipprinsip dasar ai serta pengenalan metodemetode yang ada pada domain ai metode penelitian ai mampu menggunakan analisis dan statistik yang terdapat pada domain  artificial intelligence untuk melakukan pemodelan ai dan analisis masalah pada pembelajaran individu  peserta akan mengikuti pembelajaran tentang konsepkonsep analisis dan statistik dalam domain ai  yaitu data science  computer vision dan natural language proccessing siklus projekai mampu merancang dan mengimplementasikan ai cycle project  metodemetode yang ada pada domain ai  dalam kehidupan seharihari pada pembelajaran individu  peserta akan mengikuti pembelajaran tentang frame

In [31]:
print_description_clean(2)

soft skill mampu menjelaskan pengertian komunikasi interpersonal yang baik mampu menentukan penyelesaian masalah yang paling tepat dari berbagai alternatif pemecahan masalah yang ditemukan mampu menjalankan suatu komitmen secara bersama agar bisa mencapai tujuan yang sama pula mampu memahami pengertian  tujuan dan prinsipprinsip manajemen waktu serta dapat menentukan urutan prioritas mampu menjelaskan lingkup kreativitas  pola dan proses berfikir kreatif untuk menemukan cara pemecahan masalah dan pengambilan keputusan pada pembelajaran individu  peserta akan mengikuti pembelajaran dengan mendengarkan tutorial dan kuliah dari pengajarmateri ini mengajarkan komunikasi interpersonal  communication interpersonal   problem solving  kerjasama tim  teamwork   manajemen waktu  creative thinking ui ux mampu mengimplementasikan desain prototype di figma menuju flutter tutorial dalam bentuk presentasi dan diskusi mahasiswa membaca materi  mengikuti penjelasan mentor  dan mengerjakan tugastugas se

In [32]:
print_description(8)

Membantu mengurus day-to-day operational Finance Team. Mampu mengelola dan mengurus kegiatan operational pada Finance Team. Mengelola dan membantu proses audit. Mampu mengelola dan membantu proses audit AwanTunai. Membantu mengecek dan mengelola dokumen finansial. Mampu membantu proses pengecekan dan pengelolan dokumen finansial yang dibutuhkan AwanTunai. Membantu mengelola annual report. Mampu membantu proses pengelolaan pada annual financial report AwanTunai. Mengelola dan membantu proses fundraising. 
Nama: Accounting PT Simplefi Teknologi Indonesia
Alamat: Tokopedia Tower Ciputra World 2 Lt.37 Jl. Prof. Dr. Satrio No. Kav. 11 RT.3/ RW.3, Karet Semanggi, Jakarta Selatan 12950


## 5. TF-IDF & Cosine Similarity

In [33]:
df.set_index('Nama Perusahaan', inplace=True)
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df['modulpembelajaran_clean'])
cos_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cos_sim

array([[1.        , 0.03704297, 0.04613894, 0.01735909, 0.02723883,
        0.0297549 , 0.02463625, 0.00984473, 0.01256141, 0.01329276,
        0.        ],
       [0.03704297, 1.        , 0.14342579, 0.10846177, 0.06537365,
        0.10415748, 0.06312342, 0.06267958, 0.03777654, 0.03058954,
        0.02574608],
       [0.04613894, 0.14342579, 1.        , 0.12328919, 0.03921264,
        0.0707316 , 0.08933779, 0.06952709, 0.04296582, 0.03008774,
        0.01653127],
       [0.01735909, 0.10846177, 0.12328919, 1.        , 0.05330909,
        0.07208525, 0.07217588, 0.07019156, 0.06126576, 0.023959  ,
        0.02211708],
       [0.02723883, 0.06537365, 0.03921264, 0.05330909, 1.        ,
        0.02950429, 0.03974751, 0.03166633, 0.0252055 , 0.01447374,
        0.00318317],
       [0.0297549 , 0.10415748, 0.0707316 , 0.07208525, 0.02950429,
        1.        , 0.04188213, 0.10396791, 0.0266325 , 0.03664696,
        0.00837451],
       [0.02463625, 0.06312342, 0.08933779, 0.07217588, 0.

In [34]:
# Set index utama di kolom 'name'
indices = pd.Series(df.index)
indices[:50]

0                                      Wulan Nur Afifah
1                  AI 4 Jobs PT Orbit Ventura Indonesia
2     Akademi Mobile App Developer PT Nurul Fikri Ci...
3     Cloud Administrator PT MariBelajar Indonesia C...
4     Bisnis Kecerdasan Artifisial (AI-HUSTLER) PT B...
5              Data Analytics PT Zona Edukasi Nusantara
6     01BD - Systems Analyst PT. IlmuKomputerCom Bra...
7                             2D Artist PT GIT SOLUTION
8            Accounting PT Simplefi Teknologi Indonesia
9     Data Management & Analytics Projects - Data En...
10     AI & Blockchain Infrastructure Specialist, Dikti
Name: Nama Perusahaan, dtype: object

## 6. Modelling

In [35]:
def recommendations(name, cos_sim = cos_sim):
    
    recommended_modul = []
    
    # Mengambil nama hotel berdasarkan variabel indicies
    idx = indices[indices == name].index[0]

    # Membuat series berdasarkan skor kesamaan
    score_series = pd.Series(cos_sim[idx]).sort_values(ascending = False)

    # mengambil index dan dibuat 10 baris rekomendasi terbaik
    top_10_indexes = list(score_series.iloc[1:5].index)
    
    for i in top_10_indexes:
        recommended_modul.append(list(df.index)[i])
        
    return recommended_modul

## 7. Prediksi

In [36]:
recommendations('AI 4 Jobs PT Orbit Ventura Indonesia')

['Akademi Mobile App Developer PT Nurul Fikri Cipta Inovasi',
 'Cloud Administrator PT MariBelajar Indonesia Cerdas',
 'Data Analytics PT Zona Edukasi Nusantara',
 'Bisnis Kecerdasan Artifisial (AI-HUSTLER) PT BISA ARTIFISIAL INDONESIA']

In [37]:
recommendations("2D Artist PT GIT SOLUTION")

['Data Analytics PT Zona Edukasi Nusantara',
 '01BD - Systems Analyst PT. IlmuKomputerCom Braindevs Sistema',
 'Cloud Administrator PT MariBelajar Indonesia Cerdas',
 'Akademi Mobile App Developer PT Nurul Fikri Cipta Inovasi']

In [38]:
recommendations("Wulan Nur Afifah")

['Akademi Mobile App Developer PT Nurul Fikri Cipta Inovasi',
 'AI 4 Jobs PT Orbit Ventura Indonesia',
 'Data Analytics PT Zona Edukasi Nusantara',
 'Bisnis Kecerdasan Artifisial (AI-HUSTLER) PT BISA ARTIFISIAL INDONESIA']

# Selesai...